In [14]:
import requests
import pandas as pd
import time



# کلید API خودت رو اینجا بذار
API_KEY = "Wxg7LKfYiUp57euAtj9tiwhYzH9SQCjr"
headers = {"X-DUNE-API-KEY": API_KEY}

# مرحله 1: ایجاد کوئری (Execution)
sql = """
SELECT
blockchain,
project as "dex",
token_pair as "pool",
project_contract_address as "pool_id",
block_date as "date",
round(avg(ABS(1 - (token_bought_amount / token_sold_amount))) * 100,4) as "Spread",
count(tx_hash) as "swap_count",
SUM(token_bought_amount + token_sold_amount) as "volume",
avg(token_sold_amount) as "order_size"
FROM dex.trades
where block_date BETWEEN DATE '2023-05-01' AND DATE '2023-06-07'
and blockchain ='ethereum'
and project = 'curve'
and token_sold_amount > 100
group by 1,2,3,4,5
having  count(tx_hash) > 10;
"""


# اجرا
execution_url = "https://api.dune.com/api/v1/query/execute"
response = requests.post(execution_url, headers=headers, json={"sql": sql})
execution = response.json()

if "execution_id" not in execution:
    raise Exception(f"❌ Execution failed: {execution}")

execution_id = execution["execution_id"]

# منتظر اجرا بمون
status_url = f"https://api.dune.com/api/v1/execution/{execution_id}/status"
while True:
    status_resp = requests.get(status_url, headers=headers).json()
    state = status_resp["state"]
    if state == "QUERY_STATE_COMPLETED":
        break
    elif state in ["QUERY_STATE_FAILED", "QUERY_STATE_CANCELLED"]:
        raise Exception("❌ Query failed or cancelled.")
    time.sleep(2)

# دریافت نتایج
results_url = f"https://api.dune.com/api/v1/execution/{execution_id}/results"
results = requests.get(results_url, headers=headers).json()
df = pd.DataFrame(results["result"]["rows"])



ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [9]:

import requests
import pandas as pd
import time

API_KEY = "Wxg7LKfYiUp57euAtj9tiwhYzH9SQCjr"
query_id = 5274291
headers = {"X-DUNE-API-KEY": API_KEY}

# اجرا کردن کوئری
execute_url = f"https://api.dune.com/api/v1/query/{query_id}/execute"
resp = requests.post(execute_url, headers=headers)
execution = resp.json()

if "execution_id" not in execution:
    raise Exception(f"❌ Execution failed: {execution}")

execution_id = execution["execution_id"]

# منتظر بمون تا کوئری کامل بشه
status_url = f"https://api.dune.com/api/v1/execution/{execution_id}/status"
while True:
    status_resp = requests.get(status_url, headers=headers).json()
    state = status_resp["state"]
    if state == "QUERY_STATE_COMPLETED":
        break
    elif state in ["QUERY_STATE_FAILED", "QUERY_STATE_CANCELLED"]:
        raise Exception("❌ Query failed or cancelled.")
    time.sleep(2)

# گرفتن نتیجه
results_url = f"https://api.dune.com/api/v1/execution/{execution_id}/results"
results = requests.get(results_url, headers=headers).json()

# تبدیل به DataFrame
df = pd.DataFrame(results["result"]["rows"])
print(df.head())

Exception: ❌ Execution failed: {'error': 'Deprecated query engine, please see: https://dune.com/docs/query/Old-Query-Engines/'}

In [15]:
from dune_client.client import DuneClient
dune = DuneClient("$Wxg7LKfYiUp57euAtj9tiwhYzH9SQCjr")
query_result = dune.get_latest_result(5273508)

2025-06-12 18:19:10,574 WARNING urllib3.connectionpool Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))': /api/v1/query/5273508/results?limit=1
2025-06-12 18:19:24,167 ERROR dune_client.models Can't build ResultsResponse from {'error': 'invalid API Key'} due to KeyError: 'execution_id'


DuneError: Can't build ResultsResponse from {'error': 'invalid API Key'}

In [ ]:
query_result